https://www.offerzen.com/blog/how-to-build-a-content-based-recommender-system-for-your-product

https://medium.com/towards-artificial-intelligence/content-based-recommender-system-4db1b3de03e7

https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

https://www.analyticsvidhya.com/blog/2015/08/beginners-guide-learn-content-based-recommender-systems/


In [41]:
import pandas as pd
import numpy as np


In [42]:
#Loading data from csv files
books = pd.read_csv('./goodbooks-10k/books.csv', encoding = "ISO-8859-1",index_col=0)
books.head()

,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
id,,,,,,,,,,,,,,,,,,,,,
1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [43]:
books.shape

(10000, 22)

In [44]:
books.columns

Index(['book_id', 'best_book_id', 'work_id', 'books_count', 'isbn', 'isbn13',
       'authors', 'original_publication_year', 'original_title', 'title',
       'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')

In [45]:
ratings = pd.read_csv('./goodbooks-10k/ratings.csv', encoding = "ISO-8859-1")
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [46]:
book_tags = pd.read_csv('./goodbooks-10k/book_tags.csv', encoding = "ISO-8859-1")
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [47]:
tags = pd.read_csv('./goodbooks-10k/tags.csv')
tags.tail()

,tag_id,tag_name
34247,34247,Ｃhildrens
34248,34248,Ｆａｖｏｒｉｔｅｓ
34249,34249,Ｍａｎｇａ
34250,34250,ＳＥＲＩＥＳ
34251,34251,ｆａｖｏｕｒｉｔｅｓ


In [48]:
joined_tags = pd.merge(book_tags, tags, left_on='tag_id', right_on='tag_id', how='inner')
joined_tags.head() 

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,2,30574,24549,to-read
2,3,30574,496107,to-read
3,5,30574,11909,to-read
4,6,30574,298,to-read


In [49]:
joined_tags.shape

(999912, 4)

In [50]:
to_read = pd.read_csv('./goodbooks-10k/to_read.csv')
to_read.head()

,user_id,book_id
0,1,112
1,1,235
2,1,533
3,1,1198
4,1,1874


TfidfVectorizer function from scikit-learn, which transforms text to feature vectors that can be used as input to estimator.

Cosine Similarity to calculate a numeric value that denotes the similarity between two books.

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [52]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')

In [53]:
tfidf_matrix = tf.fit_transform(books['authors'])
cos_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [54]:
cos_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [55]:

# Build a 1-dimensional array with book titles
titles = books['title']
indices = pd.Series(books.index, index=books['title'])

In [56]:
# Function that get book recommendations based on the cosine similarity score of book authors
def authors_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cos_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    book_indices = [i[0] for i in sim_scores]
    return titles.iloc[book_indices]

In [57]:
#Top 10 books recommended on basis of authors
authors_recommendations('The Catcher in the Rye').head(10)

id
26                 The Da Vinci Code (Robert Langdon, #2)
145                                       Deception Point
170                                      Digital Fortress
201                  The Lost Symbol (Robert Langdon, #3)
240                          Inferno (Robert Langdon, #4)
4496    Angels and Demons / The Da Vinci Code (Robert ...
729                        Hyperion (Hyperion Cantos, #1)
1484           The Fall of Hyperion (Hyperion Cantos, #2)
3246           The Rise of Endymion (Hyperion Cantos, #4)
3247                       Endymion (Hyperion Cantos, #3)
Name: title, dtype: object

### Recommend books using the tags provided to the books.

In [58]:
books_with_tags = pd.merge(books, joined_tags, left_on='book_id', right_on='goodreads_book_id', how='inner')

In [59]:
books_with_tags.head(3)

,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,...,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,goodreads_book_id,tag_id,count,tag_name
0,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,30574,11314,to-read
1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,11305,10836,fantasy
2,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,11557,50755,favorites


In [60]:
tf1 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix1 = tf1.fit_transform(books_with_tags['tag_name'].head(10000))
cos_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)

In [61]:
cos_sim1

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [62]:
# Build a 1-dimensional array with book titles
titles1 = books['title']
indices1 = pd.Series(books.index, index=books['title'])

# Function that get book recommendations based on the cosine similarity score of books tags
def tags_recommendations(title):
    idx = indices1[title]
    sim_scores = list(enumerate(cos_sim1[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    book_indices = [i[0] for i in sim_scores]
    return titles.iloc[book_indices]

In [63]:
#Top 10 books with similar tags as given book
tags_recommendations('The Catcher in the Rye').head(10)

id
111                          The Memory Keeper's Daughter
208                  The Immortal Life of Henrietta Lacks
307     The Wise Man's Fear (The Kingkiller Chronicle,...
508      Nickel and Dimed: On (Not) Getting By in America
608                                      Love You Forever
708                                        Eleven Minutes
1109    New Moon: The Complete Illustrated Movie Compa...
1308    A Court of Mist and Fury (A Court of Thorns an...
1405               The Year of the Flood  (MaddAddam, #2)
1608                      Paranormalcy (Paranormalcy, #1)
Name: title, dtype: object

### Recommendation of books using the authors and tags attributes for better results. Creating corpus of features and calculating the TF-IDF on the corpus of attributes for gettings better recommendations.

In [64]:
temp_df = books_with_tags.groupby('book_id')['tag_name'].apply(' '.join).reset_index()
temp_df.head()

,book_id,tag_name
0,1,to-read fantasy favorites currently-reading yo...
1,2,to-read fantasy favorites currently-reading yo...
2,3,to-read fantasy favorites currently-reading yo...
3,5,to-read fantasy favorites currently-reading yo...
4,6,to-read fantasy young-adult fiction harry-pott...


In [65]:
temp_df.shape

(10000, 2)

In [66]:
books = pd.merge(books, temp_df, left_on='book_id', right_on='book_id', how='inner')

In [67]:
books.head()

,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,tag_name
0,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,to-read fantasy favorites currently-reading yo...
1,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,to-read fantasy favorites currently-reading yo...
2,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,to-read fantasy favorites currently-reading yo...
3,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,to-read favorites currently-reading young-adul...
4,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,to-read favorites currently-reading young-adul...


In [68]:
books.shape

(10000, 23)

In [69]:
books['corpus'] = (pd.Series(books[['authors', 'tag_name']]
                .fillna('')
                .values.tolist()
                ).str.join(' '))

In [70]:
print(list(books['corpus'].head(1)))

['Suzanne Collins to-read fantasy favorites currently-reading young-adult fiction books-i-own owned ya series favourites re-read adventure sci-fi-fantasy all-time-favorites default my-books reread i-own audiobook 5-stars favorite-books novels fantasy-sci-fi favorite audiobooks read-more-than-once my-library ya-fantasy teen english books ya-fiction my-favorites own-it library audio young-adult-fiction novel scifi-fantasy faves favorite-series shelfari-favorites kindle romance favourite to-buy read-in-2014 ebook contemporary 5-star coming-of-age favourite-books favs action read-in-2013 read-in-2011 finished ya-books borrowed sci-fi ya-lit loved love thriller science-fiction finished-series action-adventure scifi sf book-club speculative-fiction ebooks e-book read-in-2012 read-in-2010 survival future drama reviewed suspense dystopia dystopian post-apocalyptic read-2012 dystopias dystopian-fiction distopia distopian read-2011 teen-fiction 2012-reads futuristic ya-dystopian trilogy complete

In [71]:
tf_corpus = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix_corpus = tf_corpus.fit_transform(books['corpus'])
cosine_sim_corpus = linear_kernel(tfidf_matrix_corpus, tfidf_matrix_corpus)

In [72]:
# Build a 1-dimensional array with book titles
titles = books['title']
indices = pd.Series(books.index, index=books['title'])

In [73]:
# Function that get book recommendations based on the cosine similarity score of books tags
def corpus_recommendations(title):
    idx = indices1[title]
    sim_scores = list(enumerate(cosine_sim_corpus[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    book_indices = [i[0] for i in sim_scores]
    return titles.iloc[book_indices]

corpus_recommendations("The Hobbit")

4                         The Great Gatsby
31                         Of Mice and Men
27                       Lord of the Flies
482                       Franny and Zooey
3                    To Kill a Mockingbird
128        One Flew Over the Cuckoo's Nest
781                          The Awakening
1565                           Ethan Frome
137                     The Scarlet Letter
57      The Adventures of Huckleberry Finn
714                    Death of a Salesman
594                       A Separate Peace
130                    The Grapes of Wrath
159                     Great Expectations
386                           The Crucible
264               A Tree Grows in Brooklyn
263                     The Sun Also Rises
112                               Catch-22
300                      Heart of Darkness
455               A Streetcar Named Desire
Name: title, dtype: object

In [74]:
corpus_recommendations("Twilight (Twilight, #1)")

7                   The Catcher in the Rye
4                         The Great Gatsby
31                         Of Mice and Men
27                       Lord of the Flies
130                    The Grapes of Wrath
129                The Old Man and the Sea
781                          The Awakening
467           Their Eyes Were Watching God
57      The Adventures of Huckleberry Finn
128        One Flew Over the Cuckoo's Nest
714                    Death of a Salesman
13                             Animal Farm
1565                           Ethan Frome
455               A Streetcar Named Desire
137                     The Scarlet Letter
159                     Great Expectations
353                                Beloved
594                       A Separate Peace
1160                           Cannery Row
263                     The Sun Also Rises
Name: title, dtype: object

In [78]:
corpus_recommendations("Romeo and Juliet")

243                                Sharp Objects
225                                  Dark Places
2168                                 The Grownup
2616                      The Kind Worth Killing
3311                                 Pretty Baby
1344                                Pretty Girls
810                                The Good Girl
7870                                 Second Life
60                         The Girl on the Train
444                         Before I Go to Sleep
2206    Broken Harbour (Dublin Murder Squad, #4)
2529                                   The Widow
4899                        Where They Found Her
472       In the Woods (Dublin Murder Squad, #1)
2395                     Into the Darkest Corner
6212                                  Disclaimer
8313                               Dear Daughter
3465                               What She Knew
1305                             The Silent Wife
4839                               The Ice Twins
Name: title, dtype: 

In [80]:


df_item = books[['book_id','corpus']]



In [82]:
df_item.head()

,book_id,corpus
0,2767052,Suzanne Collins to-read fantasy favorites curr...
1,3,"J.K. Rowling, Mary GrandPrÃ© to-read fantasy f..."
2,41865,Stephenie Meyer to-read fantasy favorites curr...
3,2657,Harper Lee to-read favorites currently-reading...
4,4671,F. Scott Fitzgerald to-read favorites currentl...


In [83]:
df_tags = pd.get_dummies(df_item['corpus'])

In [85]:
df_tags.shape

(10000, 10000)

In [119]:
books['corpus'] = (pd.Series(books[['authors', 'tag_name']]
                .fillna('')
                .values.tolist()
                ).str.join(' '))

In [120]:
s = books[['authors', 'tag_name']].fillna('').values.tolist()

In [121]:
from tqdm import tqdm


In [122]:
l = []
for i in s:
    l.extend(i)

In [123]:
len(l)

20000

In [124]:
l = set(l)

In [125]:
len(l)

14664

In [126]:
l = list(l)

In [127]:
df_genre = pd.get_dummies(l)

In [128]:
df_genre.shape

(14664, 14664)

In [129]:

from sklearn.feature_extraction.text import CountVectorizer

In [130]:
vect = CountVectorizer(analyzer = 'word', ngram_range = (1,2), stop_words = 'english', min_df = 0.002)

In [131]:
vect.fit(books['corpus'])
corpus_matrix = vect.transform(books['corpus'])

In [132]:
corpus_matrix.shape

(10000, 8606)

In [133]:
features = vect.get_feature_names()
features

['00',
 '01',
 '01 folklore',
 '02',
 '02 fantasy',
 '04',
 '04 realistic',
 '10',
 '100',
 '100 100',
 '100 1001',
 '100 best',
 '100 books',
 '100 essential',
 '100 lit',
 '100 modern',
 '100 novels',
 '100 time',
 '1000',
 '1000 books',
 '1000 crime',
 '1000 english',
 '1001',
 '1001 books',
 '1001 children',
 '1001 childrens',
 '1001 die',
 '1001 import',
 '1001 list',
 '1001 literature',
 '1001 read',
 '1001books',
 '11',
 '12',
 '16th',
 '16th century',
 '1700s',
 '17th',
 '17th century',
 '1800s',
 '1800s classici',
 '1800s penguin',
 '18th',
 '18th century',
 '1900s',
 '1900s 1001',
 '1900s classic',
 '1910s',
 '1920s',
 '1930s',
 '1940s',
 '1945',
 '1945 post',
 '1950s',
 '1950s detective',
 '1950s read',
 '1960s',
 '1960s drama',
 '1960s modern',
 '1970s',
 '1970s book',
 '1970s hold',
 '1970s sff',
 '1980',
 '1980s',
 '1980s narrativa',
 '1980s paperback',
 '1990',
 '1990s',
 '1990s paperback',
 '1990s read',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '19th',
 '1

In [135]:
word_count = [df_item['corpus'].str.contains('\\b'+x+'\\b', case = False).sum() for x in features]
word_count

[30,
 37,
 0,
 58,
 0,
 43,
 0,
 25,
 372,
 31,
 35,
 0,
 1,
 0,
 41,
 39,
 0,
 33,
 229,
 1,
 26,
 18,
 607,
 0,
 0,
 0,
 0,
 0,
 0,
 511,
 0,
 73,
 37,
 36,
 61,
 0,
 26,
 64,
 0,
 161,
 33,
 43,
 98,
 0,
 126,
 21,
 28,
 43,
 95,
 124,
 113,
 34,
 20,
 159,
 20,
 7,
 169,
 41,
 20,
 198,
 0,
 0,
 28,
 21,
 261,
 21,
 92,
 27,
 396,
 110,
 121,
 23,
 20,
 30,
 35,
 47,
 69,
 495,
 0,
 386,
 0,
 0,
 34,
 60,
 166,
 21,
 23,
 390,
 64,
 22,
 71,
 47,
 56,
 99,
 110,
 151,
 22,
 18,
 213,
 45,
 52,
 351,
 21,
 63,
 107,
 24,
 97,
 568,
 53,
 23,
 3,
 46,
 37,
 135,
 75,
 394,
 43,
 26,
 0,
 21,
 32,
 20,
 24,
 1003,
 25,
 28,
 8,
 39,
 124,
 24,
 31,
 0,
 26,
 20,
 36,
 204,
 0,
 40,
 47,
 20,
 1832,
 39,
 49,
 24,
 30,
 54,
 28,
 29,
 1,
 36,
 20,
 23,
 640,
 226,
 0,
 64,
 28,
 92,
 59,
 25,
 102,
 40,
 107,
 2802,
 33,
 66,
 711,
 144,
 31,
 45,
 45,
 696,
 38,
 106,
 52,
 20,
 31,
 292,
 0,
 0,
 180,
 50,
 24,
 47,
 299,
 116,
 3924,
 56,
 21,
 17,
 82,
 27,
 131,
 39,
 87,
 29,
 22

# --------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm


In [2]:
books = pd.read_csv('./goodbooks-10k/books.csv', encoding = "ISO-8859-1",index_col=0)


In [3]:
books.head()

,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
id,,,,,,,,,,,,,,,,,,,,,
1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [4]:
books.columns

Index(['book_id', 'best_book_id', 'work_id', 'books_count', 'isbn', 'isbn13',
       'authors', 'original_publication_year', 'original_title', 'title',
       'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')

In [5]:
ratings = pd.read_csv('./goodbooks-10k/ratings.csv', encoding = "ISO-8859-1")
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [6]:
book_tags = pd.read_csv('./goodbooks-10k/book_tags.csv', encoding = "ISO-8859-1")
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [7]:
tags = pd.read_csv('./goodbooks-10k/tags.csv', encoding = "ISO-8859-1")
tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [8]:
joined_tags = pd.merge(book_tags, tags, left_on='tag_id', right_on='tag_id', how='inner')
joined_tags.head() 

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,2,30574,24549,to-read
2,3,30574,496107,to-read
3,5,30574,11909,to-read
4,6,30574,298,to-read


In [9]:
joined_tags.shape

(999912, 4)

In [10]:
books_with_tags = pd.merge(books, joined_tags, left_on='book_id', right_on='goodreads_book_id', how='inner')

In [11]:
books_with_tags.head()

,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,...,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,goodreads_book_id,tag_id,count,tag_name
0,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,30574,11314,to-read
1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,11305,10836,fantasy
2,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,11557,50755,favorites
3,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,8717,35418,currently-reading
4,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,33114,25968,young-adult


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [13]:
tf1 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix1 = tf1.fit_transform(books_with_tags['tag_name'].head(10000))
cos_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)

In [14]:
temp_df = books_with_tags.groupby('book_id')['tag_name'].apply(' '.join).reset_index()
temp_df.head()

,book_id,tag_name
0,1,to-read fantasy favorites currently-reading yo...
1,2,to-read fantasy favorites currently-reading yo...
2,3,to-read fantasy favorites currently-reading yo...
3,5,to-read fantasy favorites currently-reading yo...
4,6,to-read fantasy young-adult fiction harry-pott...


In [15]:
books = pd.merge(books, temp_df, left_on='book_id', right_on='book_id', how='inner')

In [16]:
books.shape

(10000, 23)

In [17]:
book_ratings = pd.merge(books, ratings, left_on='book_id', right_on='book_id', how='inner')

In [18]:
book_ratings.head()

,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,...,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,tag_name,user_id,rating
0,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,to-read fantasy favorites currently-reading yo...,314,3
1,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,to-read fantasy favorites currently-reading yo...,588,1
2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,to-read fantasy favorites currently-reading yo...,2077,2
3,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,to-read fantasy favorites currently-reading yo...,2487,3
4,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,to-read fantasy favorites currently-reading yo...,2900,3


In [19]:
rating_df = pd.pivot_table(book_ratings, values='rating', index=['book_id'], columns = ['user_id'])

In [20]:
rating_df.head()

user_id,2,3,4,7,9,10,11,14,15,19,...,53404,53406,53408,53409,53416,53419,53420,53422,53423,53424
book_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
rating_df.sort_index(axis=1, inplace=True)

In [22]:
users_no = rating_df.columns

# --------------------------------------------------------

In [25]:
import pandas as pd
import numpy as np
from tqdm import tqdm



In [30]:
books = pd.read_csv('./goodbooks-10k/books.csv', encoding = "ISO-8859-1",index_col=0)


In [31]:
books_title = books[['book_id', 'title']].copy()
books_title.head()

,book_id,title
id,,
1,2767052,"The Hunger Games (The Hunger Games, #1)"
2,3,Harry Potter and the Sorcerer's Stone (Harry P...
3,41865,"Twilight (Twilight, #1)"
4,2657,To Kill a Mockingbird
5,4671,The Great Gatsby


In [32]:
from sklearn.feature_extraction.text import CountVectorizer

In [33]:


#initialize vectorizer
vect = CountVectorizer(analyzer = 'word', ngram_range = (1,2), stop_words = 'english', min_df = 0.002)

In [34]:


vect.fit(books_title['title'])
title_matrix = vect.transform(books_title['title'])



In [35]:


title_matrix.shape



(10000, 261)

In [36]:
#Lets find vocabulary
features = vect.get_feature_names()
features

['01',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '39',
 'adventures',
 'alex',
 'alex cross',
 'america',
 'american',
 'angel',
 'angels',
 'anita',
 'anita blake',
 'apprentice',
 'art',
 'bad',
 'batman',
 'beautiful',
 'beauty',
 'best',
 'big',
 'black',
 'blake',
 'blake vampire',
 'blood',
 'blue',
 'body',
 'bone',
 'bones',
 'book',
 'books',
 'born',
 'bosch',
 'bosch universe',
 'boy',
 'boys',
 'broken',
 'brothers',
 'case',
 'cat',
 'child',
 'children',
 'chronicles',
 'circle',
 'city',
 'club',
 'complete',
 'confessions',
 'cross',
 'cycle',
 'dance',
 'dark',
 'dark hunter',
 'darkest',
 'darkness',
 'daughter',
 'davenport',
 'dawn',
 'day',
 'days',
 'dead',
 'death',
 'death death',
 'detective',
 'devil',
 'diaries',
 'diary',
 'die',
 'discworld',
 'dog',
 'don',
 'dragon',
 'dream',
 'dreams',
 'earth',
 'empire',
 'end',
 'fall',
 'fallen',
 'family',
 'fear',
 'files',
 'food',
 'forever',
 'forgotten',
 'forgotten realms',
 'game',
 'garden',
 'girl'

In [37]:
len(features)

261

In [38]:
df_genre = pd.get_dummies()


In [40]:
df_genre.head()

,"Angels (Walsh Family, #3)","""Ø­ÙØ§ÙØ§Øª ÙØ±ØºÙÙ Ø§ÙÙ Ø³ØªÙØ§ÙÙ ""Ø­ÙØ§ÙØªÙ Ù Ø¹ ÙÙØ± Ø§ÙØ³Ø­ÙØ§ÙÙØ©",#GIRLBOSS,'Salem's Lot,"'Tis (Frank McCourt, #2)","1,000 Places to See Before You Die",1/4 Ø¬Ø±Ø§Ù,"10% Happier: How I Tamed the Voice in My Head, Reduced Stress Without Losing My Edge, and Found Self-Help That Actually Works","100 Bullets, Vol. 1: First Shot, Last Call",100 Love Sonnets,...,Ù Ø­Ø§Ù,Ù Ø®Ø·ÙØ·Ø© Ø¨Ù Ø¥Ø³Ø­Ø§Ù: Ù Ø¯ÙÙØ© Ø§ÙÙ ÙØªÙ,ÙØ§Ø¯Ù Ø§ÙØ³ÙØ§Ø±Ø§Øª,ÙØ´Øª Ú©ØªØ§Ø¨,ÙÙØ¨ØªØ§,ÙØ§Ø­Ø© Ø§ÙØºØ±ÙØ¨,ÙÙØªÙØ¨ÙØ§,Ú¤ÙØ±ØªÙØ¬Ù,ã­ã¹ãããæ©ã1 [Kisu Yorimo Hayaku 1] (Faster than a Kiss #1),ç¾å°å¥³æ¦å£«ã»ã¼ã©ã¼ã ã¼ã³æ°è£ ç 1 [BishÅjo Senshi Sailor Moon ShinsÅban 1]
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
